In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [215]:
df = pd.read_csv('../../data/raw/national-park-trails.csv')
df.head(3)

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,15610.598,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,6920.162,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,2896.812,81.9912,1,out and back,3.0,4.5,224,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i


In [216]:
np.shape(df)

(3313, 18)

# Read from SQL

In [24]:
# Connect to database, replace with RDS string in script
cn = create_engine('sqlite:///../../data/trails.db').connect()
  
# Read as dataframe
df = pd.read_sql_table('trails', cn)

# Convert `length` units

In [20]:
def yards_to_miles(df, column_name, digits):
    df[column_name] = round(df[column_name] / 1760, digits)
    return df

# Parse location
Parse latitude and longitude from `_geoloc` column.

In [217]:
# def parse_one_geoloc(geoloc):
    
#     # Remove parenthesis and only grab latitude and longitude
#     split_ls = geoloc.strip('{}').split(',')
#     geoloc_ls = [float(i.split(':')[1].strip()) for i in split_ls]
    
#     return geoloc_ls

# def parse_geoloc(df, column_name):
    
#     # Create new columns for latitude and longitude
#     geoloc_ls = [parse_one_geoloc(i) for i in df[column_name]]
#     df['lat'] = [i[0] for i in geoloc_ls]
#     df['lng'] = [i[1] for i in geoloc_ls]
    
#     # Drop the original variable
#     df = df.drop(columns=column_name)
    
#     return df

# Create cleaned dataset

In [21]:
# yards to miles
column_name = 'length'
digits = 2

In [22]:
# Convert yards to miles
df_clean = yards_to_miles(df, column_name, digits)

# Parse geoloc
# df_geoloc = parse_geoloc(df, '_geoloc')
# df_geoloc.to_csv('../../data/models/df_cleaned.csv', index=False)

In [23]:
df_clean.head()

,trail_id,name,area_name,city_name,state_name,country_name,_geoloc,popularity,length,elevation_gain,difficulty_rating,route_type,visitor_usage,avg_rating,num_reviews,features,activities,units
0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18852, 'lng': -149.63156}",24.8931,8.87,1161.8976,5,out and back,3.0,5.0,423,"['dogs-no', 'forest', 'river', 'views', 'water...","['birding', 'camping', 'hiking', 'nature-trips...",i
1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73049, 'lng': -148.91968}",18.0311,3.93,507.7968,3,out and back,1.0,4.5,260,"['dogs-no', 'forest', 'views', 'wild-flowers',...","['birding', 'camping', 'hiking', 'nature-trips...",i
2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,"{'lat': 60.18879, 'lng': -149.631}",17.7821,1.65,81.9912,1,out and back,3.0,4.5,224,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i
3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73661, 'lng': -148.915}",16.2674,1.92,119.7864,1,loop,2.0,4.5,237,"['dogs-no', 'forest', 'lake', 'kids', 'views',...","['birding', 'hiking', 'nature-trips', 'trail-r...",i
4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,"{'lat': 63.73319, 'lng': -148.89682}",12.5935,16.92,1124.7120,5,out and back,1.0,4.5,110,"['dogs-no', 'lake', 'views', 'wild-flowers', '...","['birding', 'fishing', 'hiking', 'nature-trips...",i
